## Analysis
- Seasons of interest - June, July, August, September.
- Districts of interest - Kolhapur, Latur

- A new LSTM model to predict rainfall.

### Import libraries

In [17]:
import warnings
warnings.filterwarnings('ignore')

import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.patches as mpatches 
from matplotlib.collections import PatchCollection
import plotly.figure_factory as ff
from IPython.display import HTML, display
from IPython.core import display as ICD
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import Artificial_Neural_Networks as ANN
import ARIMA

import math
from itertools import groupby
%matplotlib inline
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import plot_model
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential
from keras.layers.merge import concatenate
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import model_from_json

from importlib import reload
import itertools

### Useful functions

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

def calculate_performance(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    return round(mse, 3), round(mae, 3), round(mape, 3), round(rmse, 3)

### Dataset

In [4]:
PATH = 'Dataset/rainfall_data_1901_to_2002.xlsx'
data = pd.read_excel(PATH)

### Preprocess data

In [5]:
data = data.drop(columns='vlookup')
data = data[data['Year'].notnull()]
data['Year'] = data.Year.astype('int')
data.index = range(len(data))

m_data = data[data['State'] == 'Maharashtra']
m_data = m_data.drop(columns='State')

districts = m_data.District.unique()
years = list(range(1901, 2003))
months = data.columns[3:]
year_month = [str(year) + '_' + month for year in years for month in months]
dates = pd.date_range(start='1901-01', freq='MS', periods=len(years)*12)

maharashtra_data = pd.DataFrame({'Year_Month': year_month})
maharashtra_data['Date'] = dates
maharashtra_data[['Year', 'Month']] = maharashtra_data['Year_Month'].str.split('_', n=1, expand=True)
maharashtra_data = maharashtra_data.drop(columns=['Year_Month'])

for district in districts:
    df = m_data[m_data.District == district].drop(columns=['District', 'Year'])
    df = df.as_matrix().reshape((len(years) * len(months), 1))[:,0]
    maharashtra_data[district] = df

maharashtra_data.head()

,Date,Year,Month,Ahmadnagar,Akola,Amravati,Aurangabad,Bhandara,Bid,Buldana,...,Nashik,Osmanabad,Parbhani,Pune,Sangli,Satara,Solapur,Wardha,Washim,Yavatmal
0,1901-01-01,1901,Jan,2.510,34.202,35.651,10.922,23.397,16.647,31.455,...,5.063,15.500,33.207,0.922,0.138,0.197,4.496,45.050,32.485,22.031
1,1901-02-01,1901,Feb,11.489,1.099,6.822,4.362,63.844,1.916,0.823,...,1.609,2.784,4.997,7.195,0.537,0.525,3.667,28.220,3.607,16.198
2,1901-03-01,1901,Mar,11.325,30.002,36.103,25.161,33.563,27.287,28.448,...,11.196,11.333,31.625,5.105,13.090,9.566,10.330,26.398,26.557,29.318
3,1901-04-01,1901,Apr,33.931,10.248,10.636,12.714,61.560,33.211,13.902,...,7.838,34.814,41.941,35.949,50.077,30.110,59.854,29.228,18.127,30.521
4,1901-05-01,1901,May,30.401,2.891,4.173,34.244,13.665,59.027,9.397,...,7.475,52.792,31.794,36.650,78.994,65.226,50.892,8.619,6.700,13.193


In [6]:
m_data = maharashtra_data.copy()

In [7]:
m_data.head()

,Date,Year,Month,Ahmadnagar,Akola,Amravati,Aurangabad,Bhandara,Bid,Buldana,...,Nashik,Osmanabad,Parbhani,Pune,Sangli,Satara,Solapur,Wardha,Washim,Yavatmal
0,1901-01-01,1901,Jan,2.510,34.202,35.651,10.922,23.397,16.647,31.455,...,5.063,15.500,33.207,0.922,0.138,0.197,4.496,45.050,32.485,22.031
1,1901-02-01,1901,Feb,11.489,1.099,6.822,4.362,63.844,1.916,0.823,...,1.609,2.784,4.997,7.195,0.537,0.525,3.667,28.220,3.607,16.198
2,1901-03-01,1901,Mar,11.325,30.002,36.103,25.161,33.563,27.287,28.448,...,11.196,11.333,31.625,5.105,13.090,9.566,10.330,26.398,26.557,29.318
3,1901-04-01,1901,Apr,33.931,10.248,10.636,12.714,61.560,33.211,13.902,...,7.838,34.814,41.941,35.949,50.077,30.110,59.854,29.228,18.127,30.521
4,1901-05-01,1901,May,30.401,2.891,4.173,34.244,13.665,59.027,9.397,...,7.475,52.792,31.794,36.650,78.994,65.226,50.892,8.619,6.700,13.193


### Useful functions

In [8]:
def get_combinations(parameters):
    return list(itertools.product(*parameters))

In [9]:
def get_latest_file(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

In [10]:
def LSTM_model(num_of_previous_months, hidden_nodes_months, 
               num_of_previous_years, hidden_nodes_years, output_nodes=1):
    
    visible1 = Input((num_of_previous_months, 1))
#     visible1 = Input((1, num_of_previous_months))
    extract1 = LSTM(hidden_nodes_months, activation='relu')(visible1)

    visible2 = Input((num_of_previous_years, 1))
#     visible2 = Input((1, num_of_previous_years))
    extract2 = LSTM(hidden_nodes_years, activation='relu')(visible2)

    merge = concatenate([extract1, extract2])
    output = Dense(output_nodes)(merge)
    
    model = Model(inputs = [visible1, visible2], outputs = output)
    model.compile(loss='mean_squared_error', optimizer='adam')
        
    return model

In [11]:
def preprocess_data(m_data, district, month, num_of_previous_months, num_of_previous_years, future_steps):
    
#     rainfall_season_data = m_data[['Date', 'Year', 'Month'] + districts_of_interest]
    rainfall_data = m_data[['Date', 'Year', 'Month', district]]
    month_data = rainfall_data[rainfall_data.Month == month]
    
    start_year = int(rainfall_data.Year.min())
    last_year = int(rainfall_data.Year.max())
    current_year = start_year + num_of_previous_years
    month_data_index = month_data.index
    
    train_data_input_1 = []
    for index in month_data_index[num_of_previous_years:]:
        data = list(rainfall_data.iloc[index - num_of_previous_months:index][district])
        train_data_input_1.append(data)
    train_data_input_1 = np.array(train_data_input_1)
    shape = train_data_input_1.shape
    train_data_input_1 = train_data_input_1.reshape(shape[0], shape[1], 1)
    
    month_data_prep = list(month_data[district])
    train_data_input_2 = []
    for i in range(0, len(month_data_prep) - num_of_previous_years):
        data = month_data_prep[i:i+num_of_previous_years]
        train_data_input_2.append(data)
    train_data_input_2 = np.array(train_data_input_2)
    shape = train_data_input_2.shape
    train_data_input_2 = train_data_input_2.reshape(shape[0], shape[1], 1)
    
    y_train = list(month_data.iloc[num_of_previous_years:][district])
    y_train = np.array(y_train)
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    
    date_index = list(month_data.iloc[-1*future_steps:]['Date'])
        
    return train_data_input_1, train_data_input_2, y_train, date_index

In [12]:
def split_and_train_LSTM(model, input_1, input_2, y_train_main, future_steps, epochs, batch_size):
    X_train_input_1, X_test_input_1, y_train, y_test = train_test_split(input_1, y_train_main, 
                                                                    test_size=future_steps, random_state=42)
    X_train_input_2, X_test_input_2, y_train, y_test = train_test_split(input_2, y_train_main, 
                                                                    test_size=future_steps, random_state=42)
    
    SOURCE_PATH = 'Models/'
    checkpoint = ModelCheckpoint(SOURCE_PATH + 'model-{epoch:03d}.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto')  
    
    model.fit([X_train_input_1, X_train_input_2], y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=True, validation_split=0.1, callbacks=[checkpoint])
    
    file_name = get_latest_file(SOURCE_PATH)
    model.load_weights(file_name)
    os.system('rm -rf %s/*' % SOURCE_PATH)
    
    return model, X_test_input_1, X_test_input_2, y_test

In [13]:
def predict_LSTM(model, X_test_input_1, X_test_input_2):
    y_pred = model.predict([X_test_input_1, X_test_input_2])
    return y_pred

In [14]:
def Long_Short_Term_Memory(data, district, month, num_of_prev_months, num_of_prev_years, hidden_nodes_months, hidden_nodes_years, epochs, batch_size, future_steps):
    output_nodes = 1
    model = LSTM_model(num_of_prev_months, hidden_nodes_months, num_of_prev_years, hidden_nodes_years, output_nodes)
    train_data_input_1, train_data_input_2, y_train_main, date_index = preprocess_data(m_data, district, month, num_of_prev_months, num_of_prev_years, future_steps)
    model, X_test_input_1, X_test_input_2, y_test = split_and_train_LSTM(model, train_data_input_1, train_data_input_2, y_train_main, future_steps, epochs, batch_size)
    
    y_pred = predict_LSTM(model, X_test_input_1, X_test_input_2)
    return model, y_test, y_pred, date_index

In [15]:
def find_best_method_LSTM(parameters_LSTM, m_data, district, month):
    combination_of_params = get_combinations(parameters_LSTM)
    information_LSTM = []
    iterator = 0
    
    print('LSTM - Number of combinations: ' + str(len(combination_of_params)))
    
    for param in combination_of_params:
        if (iterator+1) != len(combination_of_params):
            print(iterator+1, end=' -> ')
        else:
            print(iterator+1)
        iterator = iterator+1
        
        num_of_previous_months = param[0]
        num_of_previous_years = param[1]
        hidden_nodes_months = param[2]
        hidden_nodes_years = param[3]
        epochs = param[4]
        batch_size = param[5]
        future_steps = param[6]
        
        model_LSTM, y_test, y_pred, date_index = Long_Short_Term_Memory(m_data, district, month, num_of_previous_months, num_of_previous_years,
                  hidden_nodes_months, hidden_nodes_years, epochs, batch_size, future_steps)
        
        mse, mae, mape, rmse = calculate_performance(y_test, y_pred)
        
        y_test = list(np.reshape(y_test, (y_test.shape[0])))
        y_pred = list(np.reshape(y_pred, (y_pred.shape[0])))
        
        info = list(param) + [mse, mae, rmse, mape] + y_pred
        information_LSTM.append(info)
        
        print(y_pred)
        
    information_LSTM_df = pd.DataFrame(information_LSTM)
    indexes = [str(i) for i in list(range(1, future_steps+1))]
    information_LSTM_df.columns = ['Num of previous months', 'Num of previous years', 'Hidden nodes months', 
                                   'Hidden nodes years', 'epochs', 'batch_size', 'future_steps', 
                                   'MSE', 'MAE', 'RMSE', 'MAPE'] + indexes
    
    optimized_params = information_LSTM_df.iloc[information_LSTM_df.RMSE.argmin]
    future_steps = optimized_params.future_steps
    y_pred = optimized_params[-1*int(future_steps):]
    
    num_of_previous_months = int(optimized_params['Num of previous months'])
    hidden_nodes_months = int(optimized_params['Hidden nodes months'])
    num_of_previous_years = int(optimized_params['Num of previous years'])
    hidden_nodes_years = int(optimized_params['Hidden nodes years'])
    
    best_model = LSTM_model(num_of_previous_months, hidden_nodes_months, 
                   num_of_previous_years, hidden_nodes_years, output_nodes=1)
    
    return information_LSTM_df, optimized_params, y_test, y_pred, date_index, best_model

In [16]:
def save_information(OUTPUT_FOLDER, best_model, information_LSTM_df, optimized_params, y_test, y_pred, date_index, district, month):
    name = 'LSTM'
    information_LSTM_df.to_csv(OUTPUT_FOLDER + 'LSTM_information.csv', index=False)
    optimized_params.to_csv(OUTPUT_FOLDER + 'LSTM_optimized_values.csv')
    
    plot_model(best_model, OUTPUT_FOLDER + 'LSTM_best_fit_functional_model.png', show_shapes=True, show_layer_names=True)
    
    actual_forecast = pd.DataFrame({'Actual': y_test, 'Forecasted': y_pred})
    actual_forecast['Errors'] = actual_forecast['Actual'] - actual_forecast['Forecasted']
    actual_forecast.index = date_index
    actual_forecast.to_csv(OUTPUT_FOLDER + 'LSTM_actual_and_forecasted.csv')
    
    plt.figure(figsize=(10,5))
    plt.plot(actual_forecast.drop(columns=['Forecasted', 'Errors']), color='green', label='Actual values')
    plt.plot(actual_forecast.drop(columns=['Actual', 'Errors']), color='red', label='Forecasted values')
    plt.xlabel('Year')
    plt.ylabel('Monthly mean Precipitation')
    plt.legend(loc='best')
    plt.title(name + ' - ' + district + ' - ' + month + ' - Comaprison: Actual vs Forecasted')
    plt.savefig(OUTPUT_FOLDER + name + '_best_forecast'  + '.png')
    
    plt.figure(figsize=(10,5))
    plt.plot(actual_forecast.drop(columns=['Actual', 'Forecasted']), color='blue', label='Error: Actual - Forecasted')
    plt.xlabel('Year')
    plt.ylabel('Error')
    plt.legend(loc='best')
    plt.title(name + ' - ' + district + ' - ' + month + ' - Error: Actual - Forecasted')
    plt.savefig(OUTPUT_FOLDER + name + '_error_plot'  + '.png')

In [18]:
future_steps = 10

# number_of_previous_months, number_of_previous_years, hidden_nodes_months, hidden_nodes_years, epochs, batch_size, future_steps
parameters_LSTM = [[4,8,12], [4,6,8], [6,8,10], [5,7,8], [250], [10], [future_steps]]
# parameters_LSTM = [[4,8], [4], [3], [5], [1], [10], [future_steps]]

In [19]:
districts_of_interest = ['Kolhapur'] #Latur
months_of_interest = ['Jun', 'Jul', 'Aug', 'Sep']

In [ ]:
STORAGE_FOLDER = 'State_predictions_new_LSTM/'
if not os.path.exists(STORAGE_FOLDER):
    os.makedirs(STORAGE_FOLDER)

for district in districts_of_interest:
    for month in months_of_interest:
        information_LSTM_df, optimized_params, y_test, y_pred, date_index, best_model = find_best_method_LSTM(parameters_LSTM, m_data, district, month)
        
        output_folder = STORAGE_FOLDER + district + '_' + month + '/'
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        save_information(output_folder, best_model, information_LSTM_df, optimized_params, y_test, y_pred, date_index, district, month)

LSTM - Number of combinations: 81
1 -> [582.02014, 564.4142, 574.8406, 618.3596, 570.4928, 597.3155, 585.393, 592.723, 627.0009, 580.8357]
2 -> [444.32272, 605.73645, 714.7945, 590.724, 518.65204, 480.32852, 513.07166, 562.71, 595.00867, 598.23236]
3 -> [507.87268, 706.1761, 563.4788, 642.1676, 708.584, 610.5482, 547.67285, 544.4419, 591.8219, 641.9253]
4 -> [600.1396, 537.17847, 562.50745, 604.29755, 562.04364, 636.664, 611.1207, 598.73553, 648.1166, 569.529]
5 -> [428.88736, 613.75793, 633.2419, 603.3351, 574.0266, 557.9442, 502.3886, 531.9011, 585.5352, 527.6594]
6 -> [611.5595, 586.098, 603.0187, 647.36035, 600.40283, 566.79926, 620.56604, 633.6474, 648.66675, 608.11865]
7 -> [595.82556, 581.59344, 613.84296, 631.9412, 594.57007, 580.84845, 623.7121, 623.87463, 640.2305, 615.06604]
8 -> [657.99194, 607.62164, 578.4631, 579.5911, 632.80756, 705.4274, 639.01227, 620.9113, 672.1686, 610.20435]
9 -> [630.2344, 626.9184, 630.374, 611.8072, 629.0577, 611.66785, 635.60614, 613.2831, 614.5

75 -> [456.58865, 461.70334, 553.3656, 628.21155, 413.21237, 585.9454, 485.29382, 433.5358, 569.8811, 768.33655]
76 -> [553.0828, 414.52588, 777.29596, 510.35376, 715.48944, 573.4123, 668.5977, 635.96155, 218.11243, 512.87946]
77 -> [515.3484, 568.4178, 569.3301, 739.2087, 484.53876, 575.81903, 554.22076, 592.9468, 682.1738, 472.61066]
78 -> [467.01804, 527.6962, 681.3117, 727.2621, 657.44, 613.7065, 504.81662, 765.13696, 191.80843, 511.4627]
79 -> [489.1914, 600.4469, 373.25925, 704.6512, 416.11203, 541.10266, 368.92957, 611.61383, 60.704304, 667.7661]
80 -> [487.44064, 601.82214, 842.7296, 815.0713, 725.24426, 1283.8717, 686.1392, 804.8482, 270.2512, 685.2215]
81
[600.72314, 572.4709, 703.14105, 413.64987, 735.56995, 543.4523, 822.52576, 648.68317, 621.5739, 552.1082]
LSTM - Number of combinations: 81
1 -> [829.8246, 837.40735, 561.10284, 576.17975, 535.496, 440.31647, 851.0657, 834.85535, 690.9045, 603.8377]
2 -> [811.8252, 751.5152, 606.4401, 651.5361, 523.1666, 401.9846, 510.91443

67 -> [688.396, 793.92487, 927.7475, 652.4676, 706.19073, 634.8639, 610.1002, 547.0448, 823.14276, 515.5402]
68 -> [406.85648, 739.34814, 978.65656, 519.6775, 655.3419, 526.17883, 746.43835, 528.84106, 333.42514, 668.8272]
69 -> [547.18085, 494.29645, 810.5698, 592.50336, 644.2538, 680.1342, 694.6896, 807.11176, 579.67737, 779.756]
70 -> [407.94534, 733.9598, 757.2925, 480.27112, 835.8513, 535.8638, 883.06036, 742.66187, 643.02783, 769.8935]
71 -> [849.23804, 745.90326, 850.24835, 562.8325, 760.96967, 606.7544, 723.6957, 559.6776, 707.6015, 580.2067]
72 -> [496.2349, 655.74115, 246.75865, 341.65894, 680.6509, 581.0706, 526.0339, 712.4725, 588.7651, 483.21002]
73 -> [279.91653, 155.85915, 271.6253, 497.007, 408.5784, 350.71115, 809.8912, 367.497, 413.67923, 514.2743]
74 -> [440.87497, 810.42267, 844.908, 534.56036, 1015.2289, 477.92813, 537.48016, 590.98553, 402.33887, 854.26337]
75 -> [563.99426, 512.39996, 736.76166, 634.12317, 744.17285, 587.4769, 674.8614, 822.36975, 295.3241, 680.2

59 -> [527.1778, 376.20322, 501.68396, 383.19125, 336.2177, 438.50415, 349.7735, 419.0521, 555.8146, 234.86009]
60 -> [421.61118, 423.5773, 476.93924, 438.37695, 387.94742, 333.18198, 412.73566, 232.38168, 569.10706, 292.8825]
61 -> [239.44962, 303.1747, 274.3681, 224.1765, 381.81158, 364.2131, 158.15964, 399.5698, 725.15735, 211.50497]
62 -> [365.96225, 234.5137, 359.0623, 317.10342, 550.4692, 430.37976, 263.16373, 166.25993, 529.13306, 277.84015]
63 -> [463.56973, 393.80734, 469.45206, 400.04147, 317.91058, 482.37268, 362.29858, 478.73953, 638.3549, 315.54214]
64 -> [1.9827813, 1.9827813, 1.9827813, 1.9827813, 1.9827813, 1.9827813, 1.9827813, 1.9827813, 1.9827813, 1.9827813]
65 -> [158.31918, 435.62967, 361.44855, 218.6896, 388.44675, 210.7151, 340.7354, 387.36917, 330.38406, 301.40726]
66 -> [499.33423, 540.80096, 453.77847, 328.67392, 418.37027, 485.99738, 359.81216, 377.9015, 341.05414, 367.93582]
67 -> [363.68875, 491.82623, 549.17664, 247.5507, 520.36804, 417.1988, 497.19183, 27